# 4_1_04 — Congruencia entre ingresos netos por participación en OSC de la pareja/dependientes y remuneración declarada
| Escenario                                                                                                         | Condición                                                      | Resultado        |
| ----------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------- | ---------------- |
| Participación NO corresponde a “Pareja” ni “Dependiente económico”, o no recibe remuneración                      | En DI, casilla ≠ Pareja/Dependiente **o** remuneración = NO    | ⚪ **NO_APLICA**  |
| Participa como pareja/dependiente **y** recibe remuneración **y** el ingreso anual neto (apartado B) > 0          | `/ingresos/totalIngresosMensualesNetos/ > 0`                   | 🟢 **CUMPLE**    |
| Participa como pareja/dependiente **y** recibe remuneración **pero** el ingreso anual neto es 0, nulo o no existe | ingreso B `<= 0` o nulo                                        | 🔴 **NO_CUMPLE** |
| Información insuficiente                                                                                          | No se puede interpretar participación, remuneración o ingresos | ⚪ **SIN_DATO**   |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "4_1_04_CONGRUENCIA_INGRESOS_OSC_PAREJA_DEP"

# --- Funciones auxiliares ---

def normalizar_si_no(valor):
    """Interpreta SI/NO como True/False."""
    if valor is None:
        return None
    v = str(valor).strip().upper()
    if v in ("SI", "SÍ", "SÍ", "TRUE", "1", "VERDADERO"):
        return True
    if v in ("NO", "0", "FALSE", "FALSO"):
        return False
    return None


def normalizar_participacion(valor):
    """
    Determina qué rol participa en OSC:
    - Pareja
    - Dependiente económico
    - Declarante
    """
    if valor is None:
        return None

    v = str(valor).strip().upper()

    if "PAREJA" in v:
        return "PAREJA"
    if "DEPENDIENTE" in v:
        return "DEPENDIENTE"
    if "DECLARANTE" in v:
        return "DECLARANTE"

    return None


def extraer_valor(doc, path):
    """Extrae valor numérico tolerante a anidamientos complejos."""
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None

        if actual in (None, "", " "):
            return None

        return float(actual)
    except:
        return None


# --- Evaluación principal ---

def evaluar_metrica(doc):
    # Participante (Pareja, Dependiente, Declarante)
    participante = normalizar_participacion(
        doc.get("declaracion", {})
           .get("intereses", {})
           .get("participacionTomaDecisiones", {})
           .get("participacion")
    )

    # SI/NO recibe remuneración
    recibe_rem = normalizar_si_no(
        doc.get("declaracion", {})
           .get("intereses", {})
           .get("participacionTomaDecisiones", {})
           .get("recibeRemuneracion")
    )

    # Ingreso anual neto de pareja/dependientes — apartado B
    ingreso_b = extraer_valor(
        doc,
        "declaracion.situacionPatrimonial.ingresos.totalIngresosMensualesNetos"
    )

    # -------------------------
    #   LÓGICA
    # -------------------------

    # Datos insuficientes
    if participante is None or recibe_rem is None:
        return "SIN_DATO"

    # Si NO es pareja ni dependiente económico → NO_APLICA
    if participante not in ("PAREJA", "DEPENDIENTE"):
        return "NO_APLICA"

    # Si participa como pareja/dependiente pero NO recibe remuneración
    if recibe_rem is False:
        return "NO_APLICA"

    # Aquí participa como pareja/dependiente y recibe remuneración
    if ingreso_b is None:
        return "NO_CUMPLE"

    if ingreso_b > 0:
        return "CUMPLE"

    return "NO_CUMPLE"


# --- Procesamiento MongoDB ---

def procesar_metrica_4_1_04():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.intereses.participacionTomaDecisiones": 1,
            "declaracion.situacionPatrimonial.ingresos.totalIngresosMensualesNetos": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error doc {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(UpdateOne(
                {"_id": doc["_id"]},
                {"$set": {METRIC_ID: resultado}},
                upsert=True
            ))

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f" > Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}\n")
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except Exception:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass
        print("\nConexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_1_04()


Procesando métrica 4_1_04_CONGRUENCIA_INGRESOS_OSC_PAREJA_DEP...



c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


 > Procesados 2000 documentos...
 > Procesados 4000 documentos...
 > Procesados 6000 documentos...
 > Procesados 8000 documentos...
 > Procesados 10000 documentos...
 > Procesados 12000 documentos...
 > Procesados 14000 documentos...
 > Procesados 16000 documentos...
 > Procesados 18000 documentos...
 > Procesados 20000 documentos...
 > Procesados 22000 documentos...
 > Procesados 24000 documentos...
 > Procesados 26000 documentos...
 > Procesados 28000 documentos...
 > Procesados 30000 documentos...
 > Procesados 32000 documentos...
 > Procesados 34000 documentos...
 > Procesados 36000 documentos...
 > Procesados 38000 documentos...
 > Procesados 40000 documentos...
 > Procesados 42000 documentos...
 > Procesados 44000 documentos...
 > Procesados 46000 documentos...
 > Procesados 48000 documentos...
 > Procesados 50000 documentos...
 > Procesados 52000 documentos...
 > Procesados 54000 documentos...
 > Procesados 56000 documentos...
 > Procesados 58000 documentos...
 > Procesados 6000